In [26]:
from datetime import datetime
import mediapipe as mp
import numpy as np
import requests
import imutils
import time
import json 
import uuid
import cv2
import os

In [27]:
def obtemCodCam():
    # Caminho do arquivo que armazena o código da câmera
    filename = 'codCam.json' 

    # Condição que analisa se o arquivo está vazio
    if not(os.path.getsize(filename)): 
      
        # Gera um dicionário com um código único com chave ID_Camera
        codCam = {'ID_Camera': str(uuid.uuid4())} 

        # Armazena o dicionário no arquivo
        with open(filename, "w") as file:  
            json.dump(codCam, file)   
    
    else:

        # Lê o arquivo utilizado para armazenar o código da câmera
        with open(filename, "r") as file:   
            codCam = json.load(file)

    # Envia o dicionário com o código da câmera para o banco de dados
    r = requests.post('http://monitoraquedas.orgfree.com/novaCamera.php', data = codCam)  
    print(r.text)

    return codCam

In [28]:
def saveInfo(cod={}):
    # Caminho do arquivo que armazena o código da câmera
    filename = 'codCam.json' 

    date = datetime.now()
        
    # Cria um dicionário com o horário e a data atual
    data = {'Horario': str(date.strftime("%H:%M:%S")),'Data': str(date.strftime("%d/%m/%Y"))}  
        
    # Vincula o dicionário com código da câmera juntamente com o dicionário de data e horário
    codCam.update(data)  

    # Envia o novo dicionário para o banco de dados
    r = requests.post('http://monitoraquedas.orgfree.com/novaQueda.php', data = codCam)
    print(r.text)

In [29]:
# Obtém o código da câmera 
codCam = obtemCodCam() 

# Faz a captura momentânea da câmera
cap = cv2.VideoCapture('-Endereço do vídeo-') 

# Redimensiona o vídeo
cap.set(3, 1280) 
cap.set(4, 720)  

# Inicia as variáveis para controle de frames e fps
min_frames = 0
pTime = 0
fps = 0

# Variável de controle para não retornar a condição de queda uma vez que já tenha entrado
status = False

# Variáveis para o início da analise das imagens utilizando a biblioteca mediapipe e opencv
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

In [30]:
while True:

    try:

        # Obtem frame do vídeo
        _, img = cap.read()

        # Trata a imagem do frame redimensionando
        img = imutils.resize(img, width=900, height=1800)

        # Transforma a imagem para uma imagem RGB no qual será utilizada em outro método posterior
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Obtém a estimativa da pose do corpo humano
        results = pose.process(img)

        # Condição que analisa se a imagem foi estimada e com isso exibe 33 pontos de referência do corpo
        if results.pose_landmarks:
            mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

        # Inicia duas listas
        x = []
        y = []

        # Obtém a altura e largura da imagem
        shape = img.shape
        
        # Bloco para que, caso não encontre nenhum corpo humano, assuma a posição zero e não pare de executar o programa
        try:

            # Obtém a posição original de cada ponto uma vez que os mesmoa são normalizados pela largura e altura da imagem
            for data_point in results.pose_landmarks.landmark:
                x.append(int(data_point.x*shape[1]))
                y.append(int(data_point.y*shape[0]))
        
        except Exception as e:
            x.append(0)
            y.append(0)
        
        # Obtém as coordenadas dos pontos mais distantes e menos distantes em x e y para poder contorná-los
        x_min = np.amin(x)
        y_min = np.amin(y)
        width = np.amax(x)-x_min
        height = np.amax(y)-y_min

        # Cria uma tupla com as coordenadas
        bbox = (x_min, y_min, width, height)

        # Caso o quadrado contornado tenha a base maior que a altura e que os frames em que isso se manteve 
        # tenha passado de 100 é considerado, então, que houve queda
        if bbox[3] < bbox[2] and min_frames > 100:
            
            # Condição para que detecte que já foi identificado que houve queda
            if not(status):

                # Emite um aviso para o banco de dados de que houve queda  
                saveInfo(codCam)

                # Flag de controle
                status = True

            # Muda o layout do quadrado e também apresenta um mensagem FALL para identificar queda
            cv2.rectangle(img,bbox,(0,0,255),2)
            cv2.putText(img, "Fall", (bbox[0],bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255),2)
            cv2.line(img,(bbox[0],bbox[1]),(bbox[0]+bbox[2],bbox[1]+bbox[3]),(0,0,255),2)
            cv2.line(img,(bbox[0],bbox[1]+bbox[3]),(bbox[0]+bbox[2],bbox[1]),(0,0,255),2)
        
        # Condição para que caso a base do quadrado seja maior do que a altura porém os frames analisados ainda são menos do que 100
        elif bbox[3] < bbox[2] and not(min_frames > 100):

            # Incrementa a quantidade de frames analisados
            min_frames += 1 

            # Muda o layout do quadrado e apresenta uma mensagem WARNING de aviso
            cv2.rectangle(img,bbox,(255,225,20),2)
            cv2.putText(img, "Warning", (bbox[0],bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,225,20),2)
        
        # Condição para que caso nenhuma das condições anteriores sejam verdadeiras
        else:

            # Flag de controle de detecção de queda é reiniciada
            status = False

            # Variável para controle de frame é reiniciada
            min_frames = 0

            # Quadrado contornado retorna para a cor verde padrão 
            cv2.rectangle(img,bbox,(0,255,0),2)  
        
        # Condição para obter a quantidade de frames por segundo o vídeo está entregando
        cTime = time.time() 
        fps = 1/(cTime - pTime)
        pTime = cTime

        # Apresenta a taxa de FPS no vídeo
        cv2.putText(img, "FPS: " + str(int(fps)), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0),2)
        
        # Abre uma outra janela com a análise do frame
        cv2.imshow("Cam", img)

        # Condição para parar o program pressionando a tecla Q
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    except Exception as e:
        print(e)
        break

# Fecha a janela de análise do frame recentemente aberta
cap.release()
cv2.destroyAllWindows()